### 강의에서 사용된 파이썬 주요 기능

- 데이터 집계 기초
  - pandas.Series.value_counts: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
  - pandas.DataFrame.groupby: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
  - pandas.DataFrame.pivot_table: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html

- 이벤트 성공 여부 집계
  - pandas.DataFrame.apply: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
  - Lambda functions: https://wikidocs.net/22804

### 경기 데이터 불러오기

In [3]:
import pandas as pd

In [4]:
match_id = 2058017
match_events = pd.read_pickle(f'data/refined_events/World_Cup/{match_id}.pkl')
match_events

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2058017,263883958,1H,1.892,9598,Croatia,14943,M. Mandžukić,Pass,Simple pass,[],52.00,33.32,NaN,NaN
1,2058017,263883959,1H,3.889,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],40.56,32.64,35.36,17.68
2,2058017,263883960,1H,6.141,9598,Croatia,8287,L. Modrić,Pass,Simple pass,[Accurate],35.36,17.68,31.20,4.76
3,2058017,263883963,1H,9.227,9598,Croatia,69409,Š. Vrsaljko,Pass,Simple pass,[Accurate],31.20,4.76,11.44,23.80
4,2058017,263883964,1H,12.659,9598,Croatia,135747,D. Subašić,Pass,Launch,[Accurate],11.44,23.80,66.56,33.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2058017,263885652,2H,2978.302,9598,Croatia,3476,I. Rakitić,Pass,Simple pass,[Accurate],47.84,54.40,66.56,63.92
1468,2058017,263885653,2H,2979.085,9598,Croatia,14812,I. Perišić,Others on the ball,Touch,[],66.56,63.92,85.28,66.64
1469,2058017,263885654,2H,2983.449,9598,Croatia,14812,I. Perišić,Pass,Cross,"[Left foot, High, Not accurate]",85.28,66.64,104.00,34.00
1470,2058017,263885613,2H,2985.869,4418,France,25381,H. Lloris,Goalkeeper leaving line,Goalkeeper leaving line,[],0.00,34.00,14.56,38.76


### 데이터 집계 기초

##### (1) Series.value_counts 함수를 활용한 항목별 횟수 집계

- 팀별 슈팅 횟수

In [5]:
shot_records = match_events[
    (match_events['event_type'] == 'Shot') |
    (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
]
shot_records['team_name'].value_counts()

Croatia    14
France      8
Name: team_name, dtype: int64

- 선수별 패스 횟수

In [6]:
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
pass_records['player_name'].value_counts()

M. Brozović     93
L. Modrić       72
D. Lovren       65
I. Rakitić      63
Š. Vrsaljko     62
D. Vida         46
I. Perišić      35
P. Pogba        31
M. Mandžukić    29
R. Varane       25
I. Strinić      25
A. Griezmann    24
B. Matuidi      22
O. Giroud       21
S. Umtiti       21
L. Hernández    20
B. Pavard       20
H. Lloris       18
K. Mbappé       14
A. Rebić        14
S. N'Zonzi      14
N. Kanté        13
D. Subašić      13
A. Kramarić     10
C. Tolisso       7
N. Fekir         2
M. Pjaca         2
Name: player_name, dtype: int64

##### (2) DataFrame.groupby 함수를 활용한 항목별 연산

- 전후반 경기 시간

In [7]:
match_events.groupby('period')['time'].max()

period
1H    2905.875
2H    3002.149
Name: time, dtype: float64

- 팀별 패스 발생 및 종료 위치

In [8]:
pass_records.groupby('team_name')[['start_x', 'start_y', 'end_x', 'end_y']].mean()

,start_x,start_y,end_x,end_y
team_name,,,,
Croatia,52.436446,31.492098,58.6300,31.055909
France,43.911111,35.848413,53.2064,35.308320


- 각 이벤트 유형의 팀별 발생 횟수

In [9]:
match_events.groupby(['team_name', 'event_type'])['event_id'].count()

team_name  event_type             
Croatia    Duel                       194
           Foul                        13
           Free kick                   42
           Offside                      1
           Others on the ball          72
           Pass                       518
           Save attempt                 7
           Shot                        14
           Substitution                 4
France     Duel                       194
           Foul                        13
           Free kick                   51
           Goalkeeper leaving line      5
           Offside                      1
           Others on the ball          86
           Pass                       241
           Save attempt                 3
           Shot                         7
           Substitution                 6
Name: event_id, dtype: int64

##### (3) pandas.pivot_table 함수를 활용한 다차원 집계

- 각 이벤트 유형의 팀별 발생 횟수

In [10]:
match_events.pivot_table(values='event_id', index='event_type', columns='team_name', aggfunc='count')

team_name,Croatia,France
event_type,,
Duel,194.0,194.0
Foul,13.0,13.0
Free kick,42.0,51.0
Goalkeeper leaving line,NaN,5.0
Offside,1.0,1.0
Others on the ball,72.0,86.0
Pass,518.0,241.0
Save attempt,7.0,3.0
Shot,14.0,7.0


In [11]:
counts = match_events.pivot_table('event_id', 'event_type', 'team_name', 'count', fill_value=0)
counts.sort_values('France', ascending=False)

team_name,Croatia,France
event_type,,
Pass,518,241
Duel,194,194
Others on the ball,72,86
Free kick,42,51
Foul,13,13
Shot,14,7
Substitution,4,6
Goalkeeper leaving line,0,5
Save attempt,7,3


### 이벤트 성공 여부 집계

In [12]:
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
pass_records.head()

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2058017,263883958,1H,1.892,9598,Croatia,14943,M. Mandžukić,Pass,Simple pass,[],52.00,33.32,NaN,NaN
1,2058017,263883959,1H,3.889,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],40.56,32.64,35.36,17.68
2,2058017,263883960,1H,6.141,9598,Croatia,8287,L. Modrić,Pass,Simple pass,[Accurate],35.36,17.68,31.20,4.76
3,2058017,263883963,1H,9.227,9598,Croatia,69409,Š. Vrsaljko,Pass,Simple pass,[Accurate],31.20,4.76,11.44,23.80
4,2058017,263883964,1H,12.659,9598,Croatia,135747,D. Subašić,Pass,Launch,[Accurate],11.44,23.80,66.56,33.32


##### (1) 반복문 기반 성공 여부 판단

In [13]:
success_idx = []
for i in pass_records.index:
    tags = pass_records.at[i, 'tags']
    if 'Accurate' in tags:
        success_idx.append(i)

acc_pass_records = pass_records.loc[success_idx]
acc_pass_records

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
1,2058017,263883959,1H,3.889,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],40.56,32.64,35.36,17.68
2,2058017,263883960,1H,6.141,9598,Croatia,8287,L. Modrić,Pass,Simple pass,[Accurate],35.36,17.68,31.20,4.76
3,2058017,263883963,1H,9.227,9598,Croatia,69409,Š. Vrsaljko,Pass,Simple pass,[Accurate],31.20,4.76,11.44,23.80
4,2058017,263883964,1H,12.659,9598,Croatia,135747,D. Subašić,Pass,Launch,[Accurate],11.44,23.80,66.56,33.32
7,2058017,263883967,1H,18.364,9598,Croatia,14812,I. Perišić,Pass,Simple pass,[Accurate],65.52,54.40,53.04,56.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2058017,263885639,2H,2886.43,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],64.48,57.12,76.96,60.52
1449,2058017,263885640,2H,2888.451,9598,Croatia,69411,A. Kramarić,Pass,Simple pass,[Accurate],76.96,60.52,88.40,64.60
1453,2058017,263885642,2H,2897.301,9598,Croatia,69396,D. Vida,Pass,Simple pass,[Accurate],48.88,65.28,61.36,51.00
1454,2058017,263885644,2H,2900.174,9598,Croatia,3476,I. Rakitić,Pass,Simple pass,[Accurate],61.36,51.00,69.68,55.76


##### (2) DataFrame.apply 함수 기반 성공 여부 판단

In [14]:
def is_accurate(tags):
    return 'Accurate' in tags

In [15]:
pass_records['tags'].apply(is_accurate)

0       False
1        True
2        True
3        True
4        True
        ...  
1454     True
1464    False
1467     True
1469    False
1471    False
Name: tags, Length: 781, dtype: bool

In [16]:
acc_pass_records = pass_records[pass_records['tags'].apply(is_accurate)]
acc_pass_records

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
1,2058017,263883959,1H,3.889,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],40.56,32.64,35.36,17.68
2,2058017,263883960,1H,6.141,9598,Croatia,8287,L. Modrić,Pass,Simple pass,[Accurate],35.36,17.68,31.20,4.76
3,2058017,263883963,1H,9.227,9598,Croatia,69409,Š. Vrsaljko,Pass,Simple pass,[Accurate],31.20,4.76,11.44,23.80
4,2058017,263883964,1H,12.659,9598,Croatia,135747,D. Subašić,Pass,Launch,[Accurate],11.44,23.80,66.56,33.32
7,2058017,263883967,1H,18.364,9598,Croatia,14812,I. Perišić,Pass,Simple pass,[Accurate],65.52,54.40,53.04,56.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2058017,263885639,2H,2886.43,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],64.48,57.12,76.96,60.52
1449,2058017,263885640,2H,2888.451,9598,Croatia,69411,A. Kramarić,Pass,Simple pass,[Accurate],76.96,60.52,88.40,64.60
1453,2058017,263885642,2H,2897.301,9598,Croatia,69396,D. Vida,Pass,Simple pass,[Accurate],48.88,65.28,61.36,51.00
1454,2058017,263885644,2H,2900.174,9598,Croatia,3476,I. Rakitić,Pass,Simple pass,[Accurate],61.36,51.00,69.68,55.76


##### (3) 람다 표현식(lambda expression) 기반 성공 여부 판단

In [17]:
acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_pass_records

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
1,2058017,263883959,1H,3.889,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],40.56,32.64,35.36,17.68
2,2058017,263883960,1H,6.141,9598,Croatia,8287,L. Modrić,Pass,Simple pass,[Accurate],35.36,17.68,31.20,4.76
3,2058017,263883963,1H,9.227,9598,Croatia,69409,Š. Vrsaljko,Pass,Simple pass,[Accurate],31.20,4.76,11.44,23.80
4,2058017,263883964,1H,12.659,9598,Croatia,135747,D. Subašić,Pass,Launch,[Accurate],11.44,23.80,66.56,33.32
7,2058017,263883967,1H,18.364,9598,Croatia,14812,I. Perišić,Pass,Simple pass,[Accurate],65.52,54.40,53.04,56.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2058017,263885639,2H,2886.43,9598,Croatia,69968,M. Brozović,Pass,Simple pass,[Accurate],64.48,57.12,76.96,60.52
1449,2058017,263885640,2H,2888.451,9598,Croatia,69411,A. Kramarić,Pass,Simple pass,[Accurate],76.96,60.52,88.40,64.60
1453,2058017,263885642,2H,2897.301,9598,Croatia,69396,D. Vida,Pass,Simple pass,[Accurate],48.88,65.28,61.36,51.00
1454,2058017,263885644,2H,2900.174,9598,Croatia,3476,I. Rakitić,Pass,Simple pass,[Accurate],61.36,51.00,69.68,55.76


### 경기 통계 정리

##### (1) 패스 성공률 산출

In [18]:
total_pass_counts = pass_records['team_name'].value_counts().rename('total_passes')
acc_pass_counts = acc_pass_records['team_name'].value_counts().rename('acc_passes')
pass_counts = pd.concat([total_pass_counts, acc_pass_counts], axis=1)
pass_counts

,total_passes,acc_passes
Croatia,529,436
France,252,193


In [19]:
pass_counts['pass_accuracy'] = pass_counts['acc_passes'] / pass_counts['total_passes']
pass_counts

,total_passes,acc_passes,pass_accuracy
Croatia,529,436,0.824197
France,252,193,0.765873


##### (2) 유효 슈팅 횟수 집계

In [20]:
shot_records = match_events[
    (match_events['event_type'] == 'Shot') |
    (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
]
acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]

total_shot_counts = shot_records['team_name'].value_counts().rename('total_shots')
acc_shot_counts = acc_shot_records['team_name'].value_counts().rename('shots_on_target')
shot_counts = pd.concat([total_shot_counts, acc_shot_counts], axis=1)
shot_counts

,total_shots,shots_on_target
Croatia,14,3
France,8,6


##### (3) 득점 집계

- 득점 기록 필터링

In [21]:
match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
434,2058017,263884397,1H,1676.03,9598,Croatia,14812,I. Perišić,Shot,Shot,"[Goal, Left foot, Opportunity, Position: Goal ...",90.48,39.44,104.0,34.0
505,2058017,263884513,1H,2278.896,4418,France,3682,A. Griezmann,Free kick,Penalty,"[Goal, Left foot, Position: Goal low left, Acc...",92.56,34.68,104.0,34.0
921,2058017,263885027,2H,816.147,4418,France,7936,P. Pogba,Shot,Shot,"[Goal, Left foot, Opportunity, Position: Goal ...",88.40,37.40,104.0,34.0
1025,2058017,263885135,2H,1177.059,4418,France,353833,K. Mbappé,Shot,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",78.00,29.92,104.0,34.0
1076,2058017,263885208,2H,1389.183,9598,Croatia,14943,M. Mandžukić,Shot,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",95.68,31.96,104.0,34.0


- 자책골 기록 필터링

In [22]:
match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
309,2058017,263884274,1H,1078.761,9598,Croatia,14943,M. Mandžukić,Others on the ball,Clearance,"[Own goal, Interception, Accurate]",11.44,26.52,0.0,34.0


- 자책골 포함 양팀 득점 집계

In [23]:
team_names = match_events['team_name'].unique()
goals = dict(zip(match_events['team_name'].unique(), [0, 0]))
goals

{'Croatia': 0, 'France': 0}

In [24]:
goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
for i, event in goal_records.iterrows():
    goals[event['team_name']] += 1

own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
for i, event in own_goal_records.iterrows():
    opponent_name = [team for team in team_names if team != event['team_name']][0]
    goals[opponent_name] += 1

goals

{'Croatia': 2, 'France': 4}

##### (4) 경기 통계 정리

In [25]:
counts

team_name,Croatia,France
event_type,,
Duel,194,194
Foul,13,13
Free kick,42,51
Goalkeeper leaving line,0,5
Offside,1,1
Others on the ball,72,86
Pass,518,241
Save attempt,7,3
Shot,14,7


In [26]:
foul_counts = counts.T[['Foul', 'Offside']]
foul_counts.columns = ['fouls', 'offsides']
foul_counts

,fouls,offsides
team_name,,
Croatia,13,1
France,13,1


In [27]:
match_stats = pd.concat([pass_counts, shot_counts, foul_counts], axis=1)
match_stats

,total_passes,acc_passes,pass_accuracy,total_shots,shots_on_target,fouls,offsides
Croatia,529,436,0.824197,14,3,13,1
France,252,193,0.765873,8,6,13,1


In [28]:
pd.Series(goals)

Croatia    2
France     4
dtype: int64

In [29]:
match_stats['goals'] = pd.Series(goals)
match_stats = match_stats[[
    'goals', 'total_shots', 'shots_on_target',
    'fouls', 'offsides',
    'total_passes', 'acc_passes', 'pass_accuracy'
]]
match_stats

,goals,total_shots,shots_on_target,fouls,offsides,total_passes,acc_passes,pass_accuracy
Croatia,2,14,3,13,1,529,436,0.824197
France,4,8,6,13,1,252,193,0.765873


## Mission
2018 월드컵에서 여러분이 인상깊게 봤던 경기 하나를 선택해서, 경기 통계를 집계하여 댓글로 공유해 주세요 :)

In [61]:
# 프랑스 VS 아르헨티나 경기 분석
match_id = 2058003
match_events = pd.read_pickle(f'data/refined_events/World_Cup/{match_id}.pkl')
match_events

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2058003,261065118,1H,1.303,4418,France,3682,A. Griezmann,Pass,Simple pass,[Accurate],52.00,34.00,34.32,21.08
1,2058003,261065119,1H,5.377,4418,France,3309,R. Varane,Pass,High pass,[Accurate],34.32,21.08,72.80,17.68
2,2058003,261065120,1H,7.508,4418,France,26010,O. Giroud,Duel,Air duel,"[Lost, Not accurate]",72.80,17.68,71.76,16.32
3,2058003,261065122,1H,8.163,12274,Argentina,86722,N. Tagliafico,Duel,Air duel,"[Won, Accurate]",31.20,50.32,32.24,51.68
4,2058003,261065121,1H,8.824,4418,France,26010,O. Giroud,Foul,Foul,[],71.76,16.32,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,2058003,261066780,2H,2988.34,12274,Argentina,3314,Á. di María,Pass,High pass,[Accurate],70.72,23.12,81.12,48.96
1434,2058003,261066781,2H,2990.99,12274,Argentina,8325,S. Agüero,Pass,Simple pass,[Accurate],81.12,48.96,93.60,49.64
1435,2058003,261066783,2H,2993.545,12274,Argentina,146459,M. Meza,Pass,Cross,"[Left foot, High, Accurate]",93.60,49.64,96.72,41.48
1436,2058003,261066821,2H,2994.626,12274,Argentina,3314,Á. di María,Shot,Shot,"[Right foot, Opportunity, Position: Out low ri...",96.72,41.48,104.00,34.00


In [62]:
# 프랑스 기준, 내림차순으로 보는 경기 이벤트
counts = match_events.pivot_table('event_id', 'event_type', 'team_name', 'count', fill_value=0)
counts.sort_values('France', ascending=False)

team_name,Argentina,France
event_type,,
Pass,486,335
Duel,168,179
Others on the ball,49,68
Free kick,48,31
Foul,16,23
Substitution,6,6
Shot,9,5
Save attempt,4,4
Offside,1,0


In [47]:
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
pass_records.head()

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2058003,261065118,1H,1.303,4418,France,3682,A. Griezmann,Pass,Simple pass,[Accurate],52.00,34.00,34.32,21.08
1,2058003,261065119,1H,5.377,4418,France,3309,R. Varane,Pass,High pass,[Accurate],34.32,21.08,72.80,17.68
5,2058003,261065123,1H,45.823,12274,Argentina,90991,F. Armani,Free kick,Free kick,[Not accurate],29.12,52.36,71.76,48.28
6,2058003,261065125,1H,49.247,4418,France,7936,P. Pogba,Pass,Simple pass,[Accurate],32.24,19.72,35.36,8.84
7,2058003,261065126,1H,51.377,4418,France,353833,K. Mbappé,Pass,Simple pass,[Accurate],35.36,8.84,23.92,11.56


In [48]:
acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_pass_records

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2058003,261065118,1H,1.303,4418,France,3682,A. Griezmann,Pass,Simple pass,[Accurate],52.00,34.00,34.32,21.08
1,2058003,261065119,1H,5.377,4418,France,3309,R. Varane,Pass,High pass,[Accurate],34.32,21.08,72.80,17.68
6,2058003,261065125,1H,49.247,4418,France,7936,P. Pogba,Pass,Simple pass,[Accurate],32.24,19.72,35.36,8.84
7,2058003,261065126,1H,51.377,4418,France,353833,K. Mbappé,Pass,Simple pass,[Accurate],35.36,8.84,23.92,11.56
8,2058003,261065127,1H,54.286,4418,France,340646,B. Pavard,Pass,Simple pass,[Accurate],23.92,11.56,8.32,36.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,2058003,261066814,2H,2952.348,4418,France,25397,S. Umtiti,Free kick,Free kick,[Accurate],63.44,47.60,102.96,57.12
1432,2058003,261066779,2H,2984.332,12274,Argentina,3355,J. Mascherano,Pass,High pass,[Accurate],34.32,44.20,70.72,23.12
1433,2058003,261066780,2H,2988.34,12274,Argentina,3314,Á. di María,Pass,High pass,[Accurate],70.72,23.12,81.12,48.96
1434,2058003,261066781,2H,2990.99,12274,Argentina,8325,S. Agüero,Pass,Simple pass,[Accurate],81.12,48.96,93.60,49.64


In [63]:
# 패스 성공률 분석
total_pass_counts = pass_records['team_name'].value_counts().rename('total_passes')
acc_pass_counts = acc_pass_records['team_name'].value_counts().rename('acc_passes')
pass_counts = pd.concat([total_pass_counts, acc_pass_counts], axis=1)
pass_counts

,total_passes,acc_passes
Argentina,502,432
France,346,300


In [50]:
# 패스 정확도 분석
pass_counts['pass_accuracy'] = pass_counts['acc_passes'] / pass_counts['total_passes']
pass_counts

,total_passes,acc_passes,pass_accuracy
Argentina,502,432,0.860558
France,346,300,0.867052


In [51]:
# 전체 슈팅 횟수 및 유효슈팅 분석
shot_records = match_events[
    (match_events['event_type'] == 'Shot') |
    (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
]
acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]

total_shot_counts = shot_records['team_name'].value_counts().rename('total_shots')
acc_shot_counts = acc_shot_records['team_name'].value_counts().rename('shots_on_target')
shot_counts = pd.concat([total_shot_counts, acc_shot_counts], axis=1)
shot_counts

,total_shots,shots_on_target
Argentina,10,4
France,8,4


In [52]:
# 골 기록 분석
match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
180,2058003,261065352,1H,761.29,4418,France,3682,A. Griezmann,Free kick,Penalty,"[Goal, Left foot, Position: Goal low center, A...",92.56,35.36,104.0,34.0
624,2058003,261065890,1H,2442.184,12274,Argentina,3314,Á. di María,Shot,Shot,"[Goal, Left foot, Opportunity, Position: Goal ...",74.88,31.28,104.0,34.0
757,2058003,261066818,2H,156.249,12274,Argentina,86220,G. Mercado,Shot,Shot,"[Goal, Left foot, Interception, Opportunity, P...",96.72,30.60,104.0,34.0
864,2058003,261066161,2H,708.162,4418,France,340646,B. Pavard,Shot,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",85.28,17.00,104.0,34.0
945,2058003,261066265,2H,1088.431,4418,France,353833,K. Mbappé,Shot,Shot,"[Goal, Left foot, Opportunity, Position: Goal ...",97.76,44.88,104.0,34.0
993,2058003,261066318,2H,1369.345,4418,France,353833,K. Mbappé,Shot,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",91.52,23.80,104.0,34.0
1411,2058003,261066745,2H,2823.552,12274,Argentina,8325,S. Agüero,Shot,Shot,"[Goal, Head/body, Opportunity, Position: Goal ...",95.68,40.80,104.0,34.0


In [53]:
# 자책골 분석
match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y


In [54]:
# 경기 점수 초기화
team_names = match_events['team_name'].unique()
goals = dict(zip(match_events['team_name'].unique(), [0, 0]))
goals

{'France': 0, 'Argentina': 0}

In [55]:
# 경기 점수 분석
goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
for i, event in goal_records.iterrows():
    goals[event['team_name']] += 1

own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
for i, event in own_goal_records.iterrows():
    opponent_name = [team for team in team_names if team != event['team_name']][0]
    goals[opponent_name] += 1

goals

{'France': 4, 'Argentina': 3}

In [56]:
# 종합 경기 기록 산출
counts

team_name,Argentina,France
event_type,,
Duel,168,179
Foul,16,23
Free kick,48,31
Offside,1,0
Others on the ball,49,68
Pass,486,335
Save attempt,4,4
Shot,9,5
Substitution,6,6


In [57]:
# 파울, 오프사이드 횟수 분석
foul_counts = counts.T[['Foul', 'Offside']]
foul_counts.columns = ['fouls', 'offsides']
foul_counts

,fouls,offsides
team_name,,
Argentina,16,1
France,23,0


In [58]:
# 경기 기록 분석
match_stats = pd.concat([pass_counts, shot_counts, foul_counts], axis=1)
match_stats

,total_passes,acc_passes,pass_accuracy,total_shots,shots_on_target,fouls,offsides
Argentina,502,432,0.860558,10,4,16,1
France,346,300,0.867052,8,4,23,0


In [59]:
# 골 기록 Series로 변환
pd.Series(goals)

France       4
Argentina    3
dtype: int64

In [60]:
# 중요 이벤트 기록 분석
match_stats['goals'] = pd.Series(goals)
match_stats = match_stats[[
    'goals', 'total_shots', 'shots_on_target',
    'fouls', 'offsides',
    'total_passes', 'acc_passes', 'pass_accuracy'
]]
match_stats

,goals,total_shots,shots_on_target,fouls,offsides,total_passes,acc_passes,pass_accuracy
Argentina,3,10,4,16,1,502,432,0.860558
France,4,8,4,23,0,346,300,0.867052
